In [3]:
import cv2
import numpy as np
import joblib
from img2vec_pytorch import Img2Vec
from PIL import Image
import mediapipe as mp
import pandas as pd

# Load your pre-trained Random Forest model
model = joblib.load(r"C:\Users\hp\Desktop\detection of the bels palsy\model randomforest\modelrandomforest.pkl")


# Initialize Img2Vec
img2vec = Img2Vec()

def crop_image_from_landmarks(image):
    # Initialize FaceMesh
    mp_face_mesh = mp.solutions.face_mesh
    face_mesh = mp_face_mesh.FaceMesh(static_image_mode=True)

    # Process the image with FaceMesh
    results = face_mesh.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

    # Check if face landmarks are detected
    if results.multi_face_landmarks:
        landmarks = results.multi_face_landmarks[0]

        # Define the landmark connections (example coordinates, adjust as needed)
        data = {
            'p1': [207, 205, 36 , 203 , 98, 97, 2 , 326 , 327 , 423 , 266 , 425 , 427 , 434 , 430 , 431 , 418 , 421 ,200 , 201 , 194 , 204 , 202 , 214],
            'p2': [205, 36 , 203 , 98, 97, 2 , 326 , 327 , 423 , 266 , 425 , 427 , 434 , 430 , 431 , 418 , 421 ,200 , 201 , 194 , 204 , 202 , 214, 207]
        }

        df = pd.DataFrame(data)
        routes_idx = []

        p1 = df.iloc[0]["p1"]
        p2 = df.iloc[0]["p2"]

        # Create routes indices
        for i in range(0, df.shape[0]):
            obj = df[df["p1"] == p2]
            p1 = obj["p1"].values[0]
            p2 = obj["p2"].values[0]
            route_idx = [p1, p2]
            routes_idx.append(route_idx)

        # Extract relative coordinates for routes
        routes = []
        for source_idx, target_idx in routes_idx:
            source = landmarks.landmark[source_idx]
            target = landmarks.landmark[target_idx]
            relative_source = (int(image.shape[1] * source.x), int(image.shape[0] * source.y))
            relative_target = (int(image.shape[1] * target.x), int(image.shape[0] * target.y))
            routes.append(relative_source)
            routes.append(relative_target)

        # Create the cropped image
        cropped_img = crop_image(image, landmarks, routes)

        return cropped_img

    else:
        print("No face landmarks detected in the image.")
        return None

def crop_image(img, landmarks, routes):
    # Create the mask
    mask = np.zeros((img.shape[0], img.shape[1]), dtype=np.uint8)
    mask = cv2.fillConvexPoly(mask, np.array(routes), 1)
    mask = mask.astype(bool)

    # Get coordinates of the bounding box
    coords = np.column_stack(np.where(mask))
    top_left = coords.min(axis=0)
    bottom_right = coords.max(axis=0)

    # Extract the sub-matrix using the bounding box coordinates
    out = np.zeros_like(img)
    out[mask] = img[mask]
    cropped_img = out[top_left[0]:bottom_right[0]+1, top_left[1]:bottom_right[1]+1]

    # Create the cropped mask
    cropped_mask = mask[top_left[0]:bottom_right[0]+1, top_left[1]:bottom_right[1]+1]

    # Fill the non-masked regions in the cropped image with the original image's pixels
    for i in range(cropped_img.shape[0]):
        for j in range(cropped_img.shape[1]):
            if not cropped_mask[i, j]:
                cropped_img[i, j] = img[top_left[0] + i, top_left[1] + j]

    return cropped_img

def image_to_vector(image):
    # Convert the image to PIL format
    image_pil = Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    # Get the vector from Img2Vec
    image_vector = img2vec.get_vec(image_pil)
    return image_vector

def calculate_stroke_score(video_path, model):
    cap = cv2.VideoCapture(video_path)
    stroke_frames = 0
    total_frames = 0

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        total_frames += 1

        # Process the frame to extract the mouth part
        cropped_mouth = crop_image_from_landmarks(frame)

        # Check if the cropping was successful
        if cropped_mouth is None or cropped_mouth.size == 0:
            continue

        # Convert the cropped mouth image to a vector
        image_vector = image_to_vector(cropped_mouth)

        # Make prediction using the Random Forest model
        prediction = model.predict([image_vector])
        
        # Assuming your model's output is 'affected' for stroke face and 'non-affected' for normal face
        if prediction[0] == 'non-affected':
            stroke_frames += 1

    cap.release()

    stroke_score = stroke_frames / total_frames if total_frames > 0 else 0
    return stroke_score

# Example usage
video_path = r"C:\Users\hp\Desktop\detection of the bels palsy\mediapipe\CAAI_dataset\bels plasy video datadset\bels plasy videos dataset\belChec\cropped_41.mp4"
stroke_score = calculate_stroke_score(video_path, model)
print(f'Stroke Score: {stroke_score}')


C:\Users\hp\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\hp\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Stroke Score: 0.125
